In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

def significance_to_pvalue(z):
    """
    Convierte un valor de significancia (en sigma) a un p-value (test unidireccional).
    """
    return norm.sf(z)
def process_file(filepath, ref_psf):
    df = pd.read_csv(filepath)
    required = ['Name', 'transit', 'Significance', 'PSF']
    for col in required:
        if col not in df.columns:
            raise ValueError(f"El archivo {filepath} no contiene la columna requerida '{col}'.")
    
    data_list = []
    for idx, row in df.iterrows():
        name = row['Name']
        transit = row['transit']
        significance = row['Significance']
        psf = row['PSF']
        
        p_val = significance_to_pvalue(significance)
        pdf_val = norm.pdf(significance)
        trials_factor = (psf ** 2) / (ref_psf ** 2)
        corrected_p = min(p_val * trials_factor, 1.0)
        corrected_sig = norm.isf(corrected_p) if corrected_p < 1.0 else 0.0
        
        data_list.append((name, transit, significance, corrected_sig, p_val, corrected_p, pdf_val))
    return data_list
    """
    Lee el archivo CSV, extrae las columnas 'Name', 'transit', 'Significance' y 'PSF',
    y calcula:
      - La significancia cruda (la dada en el archivo),
      - El p-value crudo (usando norm.sf),
      - El p-value corregido (p-value * (PSF²/ref_psf²), acotado a 1.0),
      - La significancia corregida (calculada a partir del p-value corregido),
      - Y el valor PDF (usando norm.pdf).
      
    Retorna una lista de tuplas para cada fila:
      (Name, Transit, Significance, Corrected Significance, p-value, Corrected p-value, PDF)
    """
    df = pd.read_csv(filepath)
    # Verificar columnas requeridas
    required = ['Name', 'transit', 'Significance', 'PSF']
    for col in required:
        if col not in df.columns:
            raise ValueError(f"El archivo {filepath} no contiene la columna requerida '{col}'.")
    
    data_list = []
    for idx, row in df.iterrows():
        name = row['Name']
        transit = row['transit']
        significance = row['Significance']
        psf = row['PSF']
        
        # Calcular p-value crudo y el valor PDF
        p_val = significance_to_pvalue(significance)
        pdf_val = norm.pdf(significance)
        # Calcular factor de trials y p-value corregido
        trials_factor = (psf ** 2) / (ref_psf ** 2)
        corrected_p = min(p_val * trials_factor, 1.0)
        # Calcular la significancia corregida a partir del p-value corregido.
        # Si corrected_p es 1, se asume significancia corregida de 0.
        if corrected_p >= 1.0:
            corrected_sig = 0.0
        else:
            corrected_sig = norm.isf(corrected_p)
        
        data_list.append((name, transit, significance, corrected_sig, p_val, corrected_p, pdf_val))
    return data_list

def fisher_combined(pvalues):
    pvalues = np.array(pvalues)
    X2 = -2 * np.sum(np.log(pvalues))
    dof = 2 * len(pvalues)
    p_combined = chi2.sf(X2, dof)
    return X2, dof, p_combined


def generate_histograms(all_data, output_dir):
    """
    Genera y guarda dos histogramas: uno para las significancias y otro para las significancias corregidas.
    Se guardan en la carpeta especificada en output_dir.
    """
    # Extraer los valores
    significances = [row[2] for row in all_data]
    corrected_significances = [row[3] for row in all_data]
    
    # Histograma de significancias
    plt.figure()
    plt.hist(significances, bins=20, edgecolor='black')
    plt.xlabel('Significancia')
    plt.ylabel('Frecuencia')
    plt.ylim([0,5])
    plt.title('Histograma de Significancias')
    sig_hist_filename = output_dir+'corrected_significance_hist.png'
    plt.savefig(sig_hist_filename)
    plt.close()
    
    # Histograma de significancias corregidas
    plt.figure()
    bins=np.arange(0,3,0.3)
    plt.hist(corrected_significances, edgecolor='black',bins=bins)
    plt.xlabel('Significancia Corregida')
    plt.ylabel('Frecuencia')
    plt.ylim([0,5])
    plt.title('Histograma de Significancias Corregidas')
    corrected_sig_hist_filename = output_dir+'corrected_significance_hist.png'
    plt.savefig(corrected_sig_hist_filename)
    plt.close()
    
    return sig_hist_filename, corrected_sig_hist_filename

def main(file1, file2, ref_psf, output_dir='UL_TN', output_tex='fisher_results.tex'):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Procesar ambos archivos y combinar los datos
    data1 = process_file(file1, ref_psf)
    data2 = process_file(file2, ref_psf)
    all_data = data1 + data2  # Cada fila: (Name, Transit, Significance, Corr. Significance, p-value, Corr. p-value, PDF)
    
    # Obtener la lista de significancias corregidas y mostrarla como lista de Python
    corrected_significances = [row[3] for row in all_data]
    print("Lista de Significancias Corregidas:", corrected_significances)
    
    # Generar los histogramas y obtener los nombres de los archivos
    sig_hist_filename, corrected_sig_hist_filename = generate_histograms(all_data, output_dir+'/')
    sig_hist_filename, corrected_sig_hist_filename = generate_histograms(data1, output_dir+'/1st_Transit'+f'PSF_{ref_psf}')
    sig_hist_filename, corrected_sig_hist_filename = generate_histograms(data2, output_dir+'/2nd_Transit'+f'PSF_{ref_psf}')
    
    # Para el test de Fisher usamos los p-values crudos
    corrected_significances = [row[3] for row in all_data]
    corrected_pvalues = [1 - norm.cdf(sig) for sig in corrected_significances]
    
    X2, dof, p_combined = fisher_combined(corrected_pvalues)
    
    # Convertir el valor de chi-cuadrado a p-value y significancia en "sigmas" usando la transformación de Wilson-Hilferty.
    z_approx = ((X2 / dof)**(1/3) - (1 - 2/(9*dof))) / np.sqrt(2/(9*dof))
    normal_p_val = norm.sf(z_approx)
    
    print("Valor de Chi-cuadrado:", X2)
    print("p-value (normal approx):", normal_p_val)
    print("Significancia (normal approx):", z_approx)
    
    critical_value = chi2.ppf(0.95, dof)
    decision = "reject" if X2 > critical_value else "do not reject"
    
    # Construir la tabla única en LaTeX con todas las columnas.
    header = ("\\textbf{GRB} & \\textbf{Transit} & \\textbf{Significance} & "
              "\\textbf{Corrected Significance} & \\textbf{p-value} & "
              "\\textbf{Corrected p-value} & \\textbf{PDF} \\\\ \\midrule\n")
    data_rows = "\n".join([
        f"{name} & {transit} & {sig:.2f} & {corr_sig:.2f} & {p_val:.3e} & {corr_p:.3e} & {pdf_val:.3e} \\\\"
        for name, transit, sig, corr_sig, p_val, corr_p, pdf_val in all_data
    ])
    event_table = f"""\\begin{{table}}[h!]
\\centering
\\resizebox{{\\textwidth}}{{!}}{{%
\\begin{{tabular}}{{l c c c c c c}}
\\toprule
{header}{data_rows}
\\bottomrule
\\end{{tabular}}%
}}
\\caption{{Lista de GRBs con sus Tránsitos, Significancias, Significancias Corregidas, p-values, p-values Corregidos y valores PDF.}}
\\end{{table}}
"""

    # Incluir las imágenes de los histogramas en el documento LaTeX
    figures_section = f"""\\begin{{figure}}[h!]
\\centering
\\includegraphics[width=0.45\\textwidth]{{{os.path.basename(sig_hist_filename)}}}
\\hfill
\\includegraphics[width=0.45\\textwidth]{{{os.path.basename(corrected_sig_hist_filename)}}}
\\caption{{Histograma de Significancias (izquierda) y de Significancias Corregidas (derecha).}}
\\end{{figure}}
"""

    conclusion = f"""\\section*{{Conclusion}}
Fisher's combined test integrates individual p-values to evaluate a global hypothesis. 
In this analysis, the test statistic obtained was $X^2 = {X2:.3f}$ with {dof} degrees of freedom. 
For a significance level of 0.05, the critical chi-square value is {critical_value:.3f}. 
Since $X^2$ is {"greater than" if X2 > critical_value else "less than or equal to"} the critical value, 
we {decision} the null hypothesis that the events are dependet.
\\newline
Additionally, the chi-square value was converted using a normal approximation:
\\newline
Normal p-value: {normal_p_val:.3e} and Normal significance: {z_approx:.2f} sigma.
"""

    latex_document = f"""\\documentclass[12pt]{{article}}
\\usepackage[utf8]{{inputenc}}
\\usepackage{{booktabs}}
\\usepackage{{geometry}}
\\usepackage{{graphicx}}
\\geometry{{a4paper, margin=1in}}
\\title{{Results of Fisher's Combined Test}}
\\author{{Sara Fraija}}
\\date{{\\today}}
\\begin{{document}}
\\maketitle
\\section{{Results}}

{event_table}

{figures_section}

{conclusion}

\\end{{document}}
"""

    # Guardar el documento LaTeX en la carpeta output_dir
    tex_output_path = os.path.join(output_dir, output_tex)
    with open(tex_output_path, 'w') as f:
        f.write(latex_document)
    print("LaTeX file generated and saved as", tex_output_path)

if __name__ == "__main__":
    spectral_index=2.07
    PSF=0.3
    file1 = f"/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates/PSF_{PSF}/alfa={spectral_index}/Coordinates_with_Max_Sig_1_0{PSF}.csv"
    file2 = f"/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates/PSF_{PSF}/alfa={spectral_index}/Coordinates_with_Max_Sig_2_0{PSF}.csv"
    main(file1, file2, ref_psf=PSF, output_dir='UL_TN', output_tex=f'fisher_results_{PSF}_{spectral_index}.tex')


Lista de Significancias Corregidas: [0.11483923236682764, 1.1562329044066577, 0.0, 0.0, 0.22454315788860987, 0.0, 1.6100000000000003, 2.3100000000000005, 0.65, 0.0, 0.58, 1.5800000000000007, 0.0, 2.5500000000000003, 2.22, 0.0, 0.66, 2.72, 1.0499999999999998, 1.0499999999999998, 1.4399999999999997, 2.06, 0.27, 1.0599999999999998, 0.38000000000000006, 0.9, 1.9300000000000006, 0.0, 0.52, 1.7500000000000004, 1.0999999999999999, 0.059999999999999984, 2.87, 0.27, 0.0, 1.0099999999999996, 0.83, 1.6000000000000003, 0.27, 0.7100000000000001, 0.0, 0.84, 0.36999999999999994, -0.14290451464856177, 1.4678098312359305, 0.3644483138821541, 1.0646464477874602, 0.511675146630907, 2.340439936799546, 0.9317313830597742, -0.3701059836132711, 1.8499999999999999, 0.6, 1.0, 0.27, 2.9000000000000004, 0.9799999999999999, 0.0, 2.21, 0.75, 0.7800000000000001, 1.12, 2.19, 2.9700000000000006, 0.7400000000000001, 2.06, 1.0200000000000002, 1.92, 1.2200000000000004, 1.23, 0.9200000000000002, 0.0, 0.9300000000000002, 

In [4]:
len( [0.11483923236682764, 1.1562329044066577, 0.0, 0.0, 0.22454315788860987, 0.0, 1.6100000000000003, 2.3100000000000005, 0.65, 0.0, 0.58, 1.5800000000000007, 0.0, 2.5500000000000003, 2.22, 0.0, 0.66, 2.72, 1.0499999999999998, 1.0499999999999998, 1.4399999999999997, 2.06, 0.27, 1.0599999999999998, 0.38000000000000006, 0.9, 1.9300000000000006, 0.0, 0.52, 1.7500000000000004, 1.0999999999999999, 0.059999999999999984, 2.87, 0.27, 0.0, 1.0099999999999996, 0.83, 1.6000000000000003, 0.27, 0.7100000000000001, 0.0, 0.84, 0.36999999999999994, -0.14290451464856177, 1.4678098312359305, 0.3644483138821541, 1.0646464477874602, 0.511675146630907, 2.340439936799546, 0.9317313830597742, -0.3701059836132711, 1.8499999999999999, 0.6, 1.0, 0.27, 2.9000000000000004, 0.9799999999999999, 0.0, 2.21, 0.75, 0.7800000000000001, 1.12, 2.19, 2.9700000000000006, 0.7400000000000001, 2.06, 1.0200000000000002, 1.92, 1.2200000000000004, 1.23, 0.9200000000000002, 0.0, 0.9300000000000002, 2.0200000000000005, 2.1799999999999997, 1.7699999999999998, 0.52, 3.4499999999999997, 0.0, 2.2000000000000006, 1.2200000000000004, 0.0, 1.2499999999999993, 1.2600000000000002, 2.4799999999999995, 2.51, 1.9500000000000002, 0.55, 1.2200000000000004, 0.0, 1.3000000000000003]
)

91

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

def significance_to_pvalue(z):
    return 1 - norm.cdf(z)

def process_file(filepath, ref_psf=0.3):
    df = pd.read_csv(filepath)
    required = ['Name', 'transit', 'Significance', 'PSF']
    for col in required:
        if col not in df.columns:
            raise ValueError(f"El archivo {filepath} no contiene la columna requerida '{col}'.")
    
    data_list = []
    for idx, row in df.iterrows():
        name = row['Name']
        transit = row['transit']
        significance = row['Significance']
        psf = row['PSF']
        
        p_val = significance_to_pvalue(significance)
        pdf_val = norm.pdf(significance)
        trials_factor = (psf ** 2) / (ref_psf ** 2)
        corrected_p = min(p_val * trials_factor, 1.0)
        corrected_sig = norm.isf(corrected_p) if corrected_p < 1.0 else 0.0
        
        data_list.append((name, transit, significance, corrected_sig, p_val, corrected_p, pdf_val))
    return data_list

def fisher_combined(pvalues):
    pvalues = np.array(pvalues)
    X2 = -2 * np.sum(np.log(pvalues))
    dof = 2 * len(pvalues)
    p_combined = chi2.sf(X2, dof)
    return X2, dof, p_combined

def generate_histograms(all_data, output_dir):
    significances = [row[2] for row in all_data]
    corrected_significances = [row[3] for row in all_data]
    
    plt.figure()
    plt.hist(significances, bins=20, edgecolor='black')
    plt.xlabel('Significancia')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Significancias')
    plt.savefig(os.path.join(output_dir, 'significance_hist.png'))
    plt.close()
    
    plt.figure()
    bins = np.arange(0, 3, 0.3)
    plt.hist(corrected_significances, edgecolor='black', bins=bins)
    plt.xlabel('Significancia Corregida')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Significancias Corregidas')
    plt.savefig(os.path.join(output_dir, 'corrected_significance_hist.png'))
    plt.close()

def main(file1, file2, ref_psf=0.3, output_dir='UL_TN', output_tex='fisher_results.tex'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    data1 = process_file(file1, ref_psf)
    data2 = process_file(file2, ref_psf)
    all_data = data1 + data2  
    
    generate_histograms(all_data, output_dir)
    
    corrected_significances = [row[3] for row in all_data]
    corrected_pvalues = [1 - norm.cdf(sig) for sig in corrected_significances]
    
    X2, dof, p_combined = fisher_combined(corrected_pvalues)
    
    z_approx = ((X2 / dof)**(1/3) - (1 - 2/(9*dof))) / np.sqrt(2/(9*dof))
    normal_p_val = norm.sf(z_approx)
    
    critical_value = chi2.ppf(0.95, dof)
    decision = "reject" if X2 > critical_value else "do not reject"
    
    tex_output_path = os.path.join(output_dir, output_tex)
    with open(tex_output_path, 'w') as f:
        f.write(f"""\documentclass[12pt]{{article}}
\usepackage[utf8]{{inputenc}}
\usepackage{{booktabs}}
\usepackage{{geometry}}
\usepackage{{graphicx}}
\geometry{{a4paper, margin=1in}}
\title{{Results of Fisher's Combined Test}}
\author{{Sara Fraija}}
\date{{\today}}
\begin{{document}}
\maketitle
\section{{Results}}

Fisher's Combined Test:\\
$\chi^2$: {X2:.3f}\\
DOF: {dof}\\
Combined p-value: {p_combined:.3e}\\
Significance (normal approx): {z_approx:.2f} sigma\\
Decision: {decision}

\begin{{figure}}[h!]
\centering
\includegraphics[width=0.45\textwidth]{{significance_hist.png}}
\hfill
\includegraphics[width=0.45\textwidth]{{corrected_significance_hist.png}}
\caption{{Histograma de Significancias y Significancias Corregidas.}}
\end{{figure}}

\end{{document}}""")
    print("LaTeX file generated and saved as", tex_output_path)


if __name__ == "__main__":
    spectral_index=2.07
    PSF=0.15
    file1 = f"/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates/PSF_{PSF}/alfa={spectral_index}/Coordinates_with_Max_Sig_1_0{PSF}.csv"
    file2 = f"/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates/PSF_{PSF}/alfa={spectral_index}/Coordinates_with_Max_Sig_2_0{PSF}.csv"
    main(file1, file2, ref_psf=PSF, output_dir='UL_TN', output_tex='fisher_results_{PSF}_{spectral_index}.tex')


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 1-2: truncated \uXXXX escape (880256210.py, line 112)

In [ ]:
import scipy.stats as stats
import numpy as np
Sig=np.array(Sig)
p_values=1-stats.norm.cdf(Sig)
Chi2=-2*sum(np.log(p_values))
p_value = stats.chi2.sf(Chi2, df)
sig=stats.norm.ppf(1-p_value) 
p_value,sig,Chi2

print(np.array(p_values).astype(np.float64))

[0.05369893 0.01044408 0.25784611 0.5        0.28095731 0.05705343
 0.5        0.00538615 0.01320938 0.5        0.25462691 0.0032641
 0.5        0.5        0.03215677 0.27425312 0.15865525 0.39358013
 0.00186581 0.16354306 0.5        0.01355258 0.22662735 0.21769544
 0.13135688 0.30443919 0.14351798 0.5       ]


In [ ]:
A=[1.6100000000000003, 2.3100000000000005, 0.65, 0.0, 0.58, 1.5800000000000007, 0.0, 2.5500000000000003, 2.22, 0.0, 0.66, 2.72, 0.0, 0.0, 1.8499999999999999, 0.6, 1.0, 0.27, 2.9000000000000004, 0.9799999999999999, 0.0, 2.21, 0.75, 0.7800000000000001, 1.12, 0.511675146630907, 1.0646464477874602, 0.0]
len(A)

28

In [ ]:
# import os
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import scipy.stats as stats
# from scipy.stats import norm, chi2

# def significance_to_pvalue(z):
#     """
#     Convierte un valor de significancia (en sigma) a un p-value (test unidireccional).
#     """
#     return norm.sf(z)

# def process_file(filepath, ref_psf=0.3):
#     """
#     Lee el archivo CSV, extrae las columnas 'Name', 'transit', 'Significance' y 'PSF',
#     y calcula:
#       - La significancia cruda (la dada en el archivo),
#       - El p-value crudo (usando norm.sf),
#       - El p-value corregido (p-value * (PSF²/ref_psf²), acotado a 1.0),
#       - La significancia corregida (calculada a partir del p-value corregido),
#       - Y el valor PDF (usando norm.pdf).
      
#     Retorna una lista de tuplas para cada fila:
#       (Name, Transit, Significance, Corrected Significance, p-value, Corrected p-value, PDF)
#     """
#     df = pd.read_csv(filepath)
#     # Verificar columnas requeridas
#     required = ['Name', 'transit', 'Significance', 'PSF']
#     for col in required:
#         if col not in df.columns:
#             raise ValueError(f"El archivo {filepath} no contiene la columna requerida '{col}'.")
    
#     data_list = []
#     for idx, row in df.iterrows():
#         name = row['Name']
#         transit = row['transit']
#         significance = row['Significance']
#         psf = row['PSF']
        
#         # Calcular p-value crudo y el valor PDF
#         p_val = significance_to_pvalue(significance)
#         pdf_val = norm.pdf(significance)
#         # Calcular factor de trials y p-value corregido
#         trials_factor = (psf ** 2) / (ref_psf ** 2)
#         corrected_p = min(p_val * trials_factor, 1.0)
#         # Calcular la significancia corregida a partir del p-value corregido.
#         # Si corrected_p es 1, se asume significancia corregida de 0.
#         if corrected_p >= 1.0:
#             corrected_sig = 0.0
#         else:
#             corrected_sig = norm.isf(corrected_p)
        
#         data_list.append((name, transit, significance, corrected_sig, p_val, corrected_p, pdf_val))
#     return data_list

# def fisher_combined(pvalues):
#     """
#     Combina los p-values usando el método de Fisher.
#     Retorna la estadística X^2, los grados de libertad y el p-value combinado.
#     """
#     pvalues = np.array(pvalues)
#     # Evitar log(0): reemplazar 0 por un valor muy pequeño
#     pvalues[pvalues == 0] = 1e-16
#     X2 = -2 * np.sum(np.log(pvalues))
#     dof = 2 * len(pvalues)
#     p_combined = chi2.sf(X2, dof)
#     return X2, dof, p_combined

# def generate_histograms(all_data, output_dir):
#     """
#     Genera y guarda dos histogramas: uno para las significancias y otro para las significancias corregidas.
#     Se guardan en la carpeta especificada en output_dir.
#     """
#     # Extraer los valores
#     significances = [row[2] for row in all_data]
#     corrected_significances = [row[3] for row in all_data]
    
#     # Histograma de significancias
#     plt.figure()
#     plt.hist(significances, bins=20, edgecolor='black')
#     plt.xlabel('Significancia')
#     plt.ylabel('Frecuencia')
#     plt.ylimit([0,5])
#     plt.title('Histograma de Significancias')
#     sig_hist_filename = os.path.join(output_dir, 'significance_hist.png')
#     plt.savefig(sig_hist_filename)
#     plt.close()
    
#     # Histograma de significancias corregidas
#     plt.figure()
#     plt.hist(corrected_significances, bins=20, edgecolor='black')
#     plt.xlabel('Significancia Corregida')
#     plt.ylabel('Frecuencia')
#     plt.title('Histograma de Significancias Corregidas')
#     corrected_sig_hist_filename = os.path.join(output_dir, 'corrected_significance_hist.png')
#     plt.savefig(corrected_sig_hist_filename)
#     plt.close()
    
#     return sig_hist_filename, corrected_sig_hist_filename

# def main(file1, file2, ref_psf=0.3, output_dir='UL_TN', output_tex='fisher_results.tex'):
#     # Crear la carpeta de salida si no existe
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     # Procesar ambos archivos y combinar los datos
#     data1 = process_file(file1, ref_psf)
#     data2 = process_file(file2, ref_psf)
#     all_data = data1 + data2  # Cada fila: (Name, Transit, Significance, Corr. Significance, p-value, Corr. p-value, PDF)
    
#     # Obtener la lista de significancias corregidas y mostrarla como lista de Python
#     corrected_significances = [row[3] for row in all_data]
#     print("Lista de Significancias Corregidas:", corrected_significances)
    
#     # Generar los histogramas y obtener los nombres de los archivos
#     sig_hist_filename, corrected_sig_hist_filename = generate_histograms(all_data, output_dir)
    
#     # Para el test de Fisher usamos los p-values crudos
#     all_pvals = [row[4] for row in all_data]
#     X2, dof, p_combined = fisher_combined(all_pvals)
    
#     # Método alternativo: Convertir el estadístico chi-cuadrado a p-value y luego a significancia normal
#     # Usando la función survival de chi2 y la función cuantil de la normal:
#     p_value_chi2 = stats.chi2.sf(X2, dof)
#     z_value = norm.ppf(1 - p_value_chi2)
    
#     print("Valor de Chi-cuadrado:", X2)
#     print("p-value (chi2):", p_value_chi2)
#     print("Significancia (normal):", z_value)
    
#     critical_value = chi2.ppf(0.95, dof)
#     decision = "reject" if X2 > critical_value else "do not reject"
    
#     # Construir la tabla única en LaTeX con todas las columnas.
#     header = ("\\textbf{GRB} & \\textbf{Transit} & \\textbf{Significance} & "
#               "\\textbf{Corrected Significance} & \\textbf{p-value} & "
#               "\\textbf{Corrected p-value} & \\textbf{PDF} \\\\ \\midrule\n")
#     data_rows = "\n".join([
#         f"{name} & {transit} & {sig:.2f} & {corr_sig:.2f} & {p_val:.3e} & {corr_p:.3e} & {pdf_val:.3e} \\\\"
#         for name, transit, sig, corr_sig, p_val, corr_p, pdf_val in all_data
#     ])
#     event_table = f"""\\begin{{table}}[h!]
# \\centering
# \\resizebox{{\\textwidth}}{{!}}{{%
# \\begin{{tabular}}{{l c c c c c c}}
# \\toprule
# {header}{data_rows}
# \\bottomrule
# \\end{{tabular}}%
# }}
# \\caption{{Lista de GRBs con sus Tránsitos, Significancias, Significancias Corregidas, p-values, p-values Corregidos y valores PDF.}}
# \\end{{table}}
# """

#     # Incluir las imágenes de los histogramas en el documento LaTeX
#     figures_section = f"""\\begin{{figure}}[h!]
# \\centering
# \\includegraphics[width=0.45\\textwidth]{{{os.path.basename(sig_hist_filename)}}}
# \\hfill
# \\includegraphics[width=0.45\\textwidth]{{{os.path.basename(corrected_sig_hist_filename)}}}
# \\caption{{Histograma de Significancias (izquierda) y de Significancias Corregidas (derecha).}}
# \\end{{figure}}
# """

#     conclusion = f"""\\section*{{Conclusion}}
# Fisher's combined test integrates individual p-values to evaluate a global hypothesis. 
# In this analysis, the test statistic obtained was $X^2 = {X2:.3f}$ with {dof} degrees of freedom. 
# For a significance level of 0.05, the critical chi-square value is {critical_value:.3f}. 
# Since $X^2$ is {"greater than" if X2 > critical_value else "less than or equal to"} the critical value, 
# we {decision} the null hypothesis that the events are independent.
# \\newline
# Using the chi-square distribution, we calculated:
# \\newline
# p-value: {p_value_chi2:.2e} 
# \\newline
# Normal significance (z-value): {z_value:.2f} sigma.
# """

#     latex_document = f"""\\documentclass[12pt]{{article}}
# \\usepackage[utf8]{{inputenc}}
# \\usepackage{{booktabs}}
# \\usepackage{{geometry}}
# \\usepackage{{graphicx}}
# \\geometry{{a4paper, margin=1in}}
# \\title{{Results of Fisher's Combined Test}}
# \\author{{Sara Fraija}}
# \\date{{\\today}}
# \\begin{{document}}
# \\maketitle
# \\section{{Results}}

# {event_table}

# {figures_section}

# {conclusion}

# \\end{{document}}
# """

#     # Guardar el documento LaTeX en la carpeta output_dir
#     tex_output_path = os.path.join(output_dir, output_tex)
#     with open(tex_output_path, 'w') as f:
#         f.write(latex_document)
#     print("LaTeX file generated and saved as", tex_output_path)

# if __name__ == "__main__":
#     file1 = "/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates/Coordinates_with_Max_Sig_1_00.3.csv"
#     file2 = "/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates/Coordinates_with_Max_Sig_2_00.3.csv"
#     main(file1, file2, ref_psf=0.3, output_dir='UL_TN', output_tex='fisher_results.tex')


In [ ]:
e1=16.1
e2=158.7
alfa=2.07
E0=1
N0=2.07e-10
N0=3.88e-11
1.6*N0/(E0**-alfa)*(1/(-alfa+2))*(e2**(-alfa+2)-e1**(-alfa+2))                         

1.0805629053798605e-10

In [ ]:
def non_to_int2(E1, E2, flux, alpha, pivot):
    """Convierte un flujo a un valor integrado según el índice espectral."""
    A = flux / (pivot ** (-alpha))
    if alpha > 2:
        F1 = (E2 ** (2 - alpha)) / (2 - alpha)
        F2 = (E1 ** (2 - alpha)) / (2 - alpha)
        return 1.6 * A * (F1 - F2)
    elif alpha == 2:
        F3 = np.log(E2)
        F4 = np.log(E1)
        return A * (F3 - F4)
    else:
        return None
Frans=pd.read_csv('/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/data.csv')
a=non_to_int2(e1,e2,N0,alfa,1)
E=pd.read_csv('/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Energy_ranges_2.07.csv',names=['Name','E1','E2'])
UB=Frans['upperBound_2ndT'][6];E1=E['E1'][6];E2=E['E2'][6]
UL=non_to_int2(E1,E2,UB,alfa,1)

# UB=Frans['upperBound'];E1=E['E1'];E2=E['E2']
# UL=non_to_int2(E1.values,E2.values,UB.values,alfa,1)
# 
print(UL,UB,E1,E2)

9.301752845280249e-10 3.34e-10 16.1 158.7
